In [ ]:
%config Completer.use_jedi = False

In [ ]:
# Add project src to path.
import set_path

# Import modules.
import numpy as np
import time

%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import src.fusion as fusion
import src.region_growing as growing
import src.utils.las_utils as las_utils
import src.utils.ahn_utils as ahn_utils
import src.utils.clip_utils as clip_utils
from src.utils.labels import Labels
from src.utils.interpolation import FastGridInterpolator

## Set-up

In [ ]:
tilecode = '2386_9702'
#tilecode = '2397_9705'

bgt_data_file = '../datasets/bgt/custom_points_' + tilecode + '.csv'

# Load elevation data.
ahn_data_folder = '../datasets/ahn/'
ahn_reader = ahn_utils.NPZReader(data_folder=ahn_data_folder)

In [ ]:
# Load labeled and grown LAS file.
las_file = '../datasets/pointcloud/complete_' + tilecode + '.laz'
las = las_utils.read_las(las_file)

labels = las.label
points = np.vstack((las.x, las.y, las.z)).T

## Match point objects with clusters

In [ ]:
# All possible parameters:
# params = {'r_mult': 1.5,
#           'search_pad': 1.5,
#           'max_dist': 1.2,
#           'voxel_res': 0.2,
#           'seed_height': 1.75,
#           'min_height': 2.,
#           'max_r': 0.5,
#           'min_points': 500,
#           'z_min': 0.2,
#           'z_max': 2.7,
#           'label_height': 4.}

tree_params = {'seed_height': 1.75, 'min_points': 500, 'max_r': 0.5, 'label_height': 5.}
light_params = {'seed_height': 2.25, 'min_points': 400, 'max_r': 0.2, 'label_height': 5.}
sign_params = {'seed_height': 1.75, 'min_points': 200, 'max_r': 0.2, 'min_height': 1.2, 'z_max': 2., 'label_height': 3.}

# Fusers for BGT point data.
tree_fuser = fusion.BGTPointFuser(Labels.TREE, bgt_type='boom', bgt_file=bgt_data_file, ahn_reader=ahn_reader, params=tree_params)
light_fuser = fusion.BGTPointFuser(Labels.STREET_LIGHT, bgt_type='lichtmast', bgt_file=bgt_data_file, ahn_reader=ahn_reader, params=light_params)
sign_fuser = fusion.BGTPointFuser(Labels.TRAFFIC_SIGN, bgt_type='verkeersbord', bgt_file=bgt_data_file, ahn_reader=ahn_reader, params=sign_params)

In [ ]:
mask = (labels != Labels.GROUND) & (labels != Labels.BUILDING) & (labels != Labels.CAR)

start = time.time()
tree_mask = tree_fuser.get_label_mask(points, labels, mask, tilecode)
mask[tree_mask] = False
end = time.time()
print(f'Trees fused in {end-start:.2f} seconds.\n')

start = time.time()
light_mask = light_fuser.get_label_mask(points, labels, mask, tilecode)
mask[light_mask] = False
end = time.time()
print(f'Street lights fused in {end-start:.2f} seconds.\n')

start = time.time()
sign_mask = sign_fuser.get_label_mask(points, labels, mask, tilecode)
mask[sign_mask] = False
end = time.time()
print(f'Traffic signs fused in {end-start:.2f} seconds.')

In [ ]:
labels[tree_mask] = tree_fuser.get_label()
labels[light_mask] = light_fuser.get_label()
labels[sign_mask] = sign_fuser.get_label()

## Extend point objects using LCC region growing

In [ ]:
# All possible parameters:
# params = {'plane_height': REQUIRED,
#           'octree_level': 9,
#           'min_comp_size': 100,
#           'threshold': 0.5}

light_top_params = {'plane_height': 3.25, 'threshold': 0.05}
light_bottom_params = {'plane_height': 1.5, 'threshold': 0.5}

sign_top_params = {'plane_height': 1.5, 'threshold': 0.05}
sign_bottom_params = {'plane_height': 1.5, 'threshold': 0.5}

light_lcc = growing.TopBottomLCC(Labels.STREET_LIGHT, ahn_reader, 
                                 top_params=light_top_params, 
                                 bottom_params=light_bottom_params)
sign_lcc = growing.TopBottomLCC(Labels.TRAFFIC_SIGN, ahn_reader, 
                                top_params=sign_top_params, 
                                bottom_params=sign_bottom_params)

In [ ]:
start = time.time()
light_lcc_mask = light_lcc.get_label_mask(points, labels, mask, tilecode)
labels[light_lcc_mask] = light_lcc.get_label()
mask[light_lcc_mask] = False
end = time.time()
print(f'Street lights grown in {end-start:.2f} seconds.\n')

start = time.time()
sign_lcc_mask = sign_lcc.get_label_mask(points, labels, mask, tilecode)
labels[sign_lcc_mask] = sign_lcc.get_label()
mask[sign_lcc_mask] = False
end = time.time()
print(f'Traffic signs grown in {end-start:.2f} seconds.')

### Trees, experimental

In [ ]:
tree_top_params = {'plane_height': 1.75, 'octree_level': 10, 'threshold': 0.01}
tree_bottom_params = {'plane_height': 1.75, 'octree_level': 10, 'threshold': 0.8}

tree_lcc = growing.TopBottomLCC(Labels.TREE, ahn_reader, 
                                top_params=tree_top_params, 
                                bottom_params=tree_bottom_params)

start = time.time()
tree_lcc_mask = tree_lcc.get_label_mask(points, labels, mask, tilecode)
labels[tree_lcc_mask] = tree_lcc.get_label()
mask[tree_lcc_mask] = False
end = time.time()
print(f'Trees grown in {end-start:.2f} seconds.')

### Save LAS file

In [ ]:
# Save the result.
out_file = '../datasets/pointcloud/poles_custom_' + tilecode + '.laz'
las_utils.label_and_save_las(las, labels, out_file)

## Visualize results

In [ ]:
# Visualize the resulting match.
((x_min, y_max), (x_max, y_min)) = las_utils.get_bbox_from_tile_code(tilecode)

ahn_tile = ahn_reader.filter_tile(tilecode)
fast_z = FastGridInterpolator(ahn_tile['x'], ahn_tile['y'],
                              ahn_tile['ground_surface'])
points_z = fast_z(points[:, 0:2])

plane_mask = (points[:, 2] >= points_z + 1.7) & (points[:, 2] <= points_z + 1.8)
label_set = np.unique(labels[plane_mask])

fig, ax = plt.subplots(1, constrained_layout=True)

for label in label_set:
    label_mask = plane_mask & (labels == label)
    scatter = ax.scatter(points[label_mask, 0], points[label_mask, 1], marker='.', label=Labels.get_str(label))

box = patches.Rectangle((x_min, y_min), x_max-x_min, y_max-y_min, linewidth=1, linestyle='--', edgecolor='grey', fill=False)

ax.add_patch(box)
ax.set_title(tilecode)
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.axis('equal')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()